## еще можно open, promo, SchoolHoliday сделать dummy

In [86]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [79]:
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y[i] != 0:
            summ += (1-y_pred[i]/y[i])**2
    return sqrt(summ/len(y))

In [98]:
df_train = pd.read_csv('data/train_v2.csv')
df_train['Date']  = pd.to_datetime(df_train['Date'], errors='coerce')
df_train['Year'] = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month
df_train['WeekOfYear'] = df_train['Date'].dt.weekofyear
df_train['Day'] = df_train['Date'].dt.day
df_train = df_train.drop('Date', axis=1)

# df_train = df_train[df_train["Open"] != 0]
# df_train = df_train[df_train["Sales"] > 0]

df_test = pd.read_csv('data/test_v2.csv')
df_test['Date']  = pd.to_datetime(df_test['Date'], errors='coerce')
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['WeekOfYear'] = df_test['Date'].dt.weekofyear
df_test['Day'] = df_test['Date'].dt.day
df_test = df_test.drop('Date', axis=1)

df_store = pd.read_csv('data/store.csv')
df_store['CompetitionDistance'].fillna(-1,inplace=True)
df_store['CompetitionOpenSinceMonth'].fillna(-1,inplace=True)
df_store['CompetitionOpenSinceYear'].fillna(-1,inplace=True)
df_store['Promo2SinceWeek'].fillna(0,inplace=True)
df_store['Promo2SinceYear'].fillna(0,inplace=True)
df_store['PromoInterval'].fillna(0,inplace=True)

df_train_store = df_train.join(df_store.set_index('Store'), on='Store')
df_train_store['CompetitionOpen'] = 12 * (df_train_store.Year - df_train_store.CompetitionOpenSinceYear) + \
        (df_train_store.Month - df_train_store.CompetitionOpenSinceMonth)
df_train_store['CompetitionOpen'] = df_train_store.CompetitionOpen.apply(lambda x: x if x > 0 else 0)
df_train_store.loc[df_train_store.CompetitionOpenSinceYear < 0, 'CompetitionOpen'] = 0
df_train_store['PromoOpen'] = 12 * (df_train_store.Year - df_train_store.Promo2SinceYear) + \
        (df_train_store.WeekOfYear - df_train_store.Promo2SinceWeek) / 4.0
df_train_store['PromoOpen'] = df_train_store.PromoOpen.apply(lambda x: x if x > 0 else 0)
df_train_store.loc[df_train_store.Promo2SinceYear == 0, 'PromoOpen'] = 0
df_train_store.drop(['CompetitionOpenSinceYear',
                     'CompetitionOpenSinceMonth',
                     'Promo2SinceYear',
                     'Promo2SinceWeek'], axis=1, inplace=True)
df_train_store.drop(['Promo2','PromoInterval'], axis=1, inplace=True)
df_train_store.drop(['Year','Month'], axis=1, inplace=True)


df_test_store = df_test.join(df_store.set_index('Store'), on='Store')
df_test_store['CompetitionOpen'] = 12 * (df_test_store.Year - df_test_store.CompetitionOpenSinceYear) + \
        (df_test_store.Month - df_test_store.CompetitionOpenSinceMonth)
df_test_store['CompetitionOpen'] = df_test_store.CompetitionOpen.apply(lambda x: x if x > 0 else 0)
df_test_store.loc[df_test_store.CompetitionOpenSinceYear < 0, 'CompetitionOpen'] = 0
df_test_store['PromoOpen'] = 12 * (df_test_store.Year - df_test_store.Promo2SinceYear) + \
        (df_test_store.WeekOfYear - df_test_store.Promo2SinceWeek) / 4.0
df_test_store['PromoOpen'] = df_test_store.PromoOpen.apply(lambda x: x if x > 0 else 0)
df_test_store.loc[df_test_store.Promo2SinceYear == 0, 'PromoOpen'] = 0
df_test_store.drop(['CompetitionOpenSinceYear',
                     'CompetitionOpenSinceMonth',
                     'Promo2SinceYear',
                     'Promo2SinceWeek'], axis=1, inplace=True)
df_test_store.drop(['Promo2','PromoInterval'], axis=1, inplace=True)
df_test_store.drop(['Year','Month'], axis=1, inplace=True)


df = df_train_store
labels = df['Sales']
df.drop(['Sales'], axis=1, inplace=True)
df_train_store_with_out_sales = df
frames = [df_train_store_with_out_sales, df_test_store]
super_df = pd.concat(frames)

mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
super_df.StateHoliday.replace(mappings, inplace=True)
super_df.StoreType.replace(mappings, inplace=True)
super_df.Assortment.replace(mappings, inplace=True)

# scaler = MinMaxScaler() 
# scaled_values = scaler.fit_transform(super_df) 
# super_df.iloc[:,2] = scaled_values[:,2]
# super_df.iloc[:,11:14] = scaled_values[:,11:14]

super_df = pd.get_dummies(super_df, columns=['Store', 
                                                         'DayOfWeek',
                                                         'Open',
                                                         'Promo',
                                                         'SchoolHoliday',
                                                         'StateHoliday',
                                                         'WeekOfYear',
                                                         'Day',
                                                         'StoreType',
                                                         'Assortment'])

start_ind = df_train_store.shape[0]
df_train_store = super_df.iloc[:start_ind,:]
df_test_store = super_df.iloc[start_ind:,:]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [99]:
super_df.head()

,Customers,CompetitionDistance,CompetitionOpen,PromoOpen,Store_1,Store_2,Store_3,Store_4,Store_5,Store_6,...,Day_29,Day_30,Day_31,StoreType_1,StoreType_2,StoreType_3,StoreType_4,Assortment_1,Assortment_2,Assortment_3
0,616,1270.0,76.0,0.00,1,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
1,624,570.0,86.0,58.00,0,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
2,678,14130.0,97.0,45.75,0,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,1632,620.0,64.0,0.00,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
4,617,29910.0,0.0,0.00,0,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0


In [82]:
df_test_store.head()

,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpen,PromoOpen,Store_1,Store_2,Store_3,...,Day_29,Day_30,Day_31,StoreType_1,StoreType_2,StoreType_3,StoreType_4,Assortment_1,Assortment_2,Assortment_3
0,0.111850,1,1,1,0.016754,0.059163,0.000000,1,0,0,...,0,0,1,0,0,1,0,1,0,0
1,0.125957,1,1,1,0.007527,0.066378,0.895833,0,1,0,...,0,0,1,1,0,0,0,1,0,0
2,0.165457,1,1,1,0.186275,0.074315,0.725694,0,0,1,...,0,0,1,1,0,0,0,1,0,0
3,0.301894,1,1,1,0.008186,0.050505,0.000000,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,0.112656,1,1,1,0.394287,0.002165,0.000000,0,0,0,...,0,0,1,1,0,0,0,1,0,0


In [100]:
X = df_train_store.values
y = labels.values

X_test = df_test_store.values

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.85, random_state=1234)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [102]:
out_dims = [150]
bss = [2000]
num_epochs = 200
verb = 1
for out_dim in out_dims:
    for bs in bss:
        model = Sequential()
        model.add(Dense(output_dim=out_dim, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(output_dim=1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='adam')
        
        start = time.time()
        
        model.fit(X_train, y_train, 
        verbose=verb, 
        nb_epoch=num_epochs, 
        batch_size=bs,
        validation_data=(X_validation, y_validation))
        
        end = time.time()
        
        pred = model.predict(X_validation)
        accuracy = rmspe(y_validation, pred)
        print('---------//---------')
        print('output     ',out_dim)
        print('batch size ',bs)
        print('time taken ',end-start)
        print('E_out is   ',accuracy)
        print()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=1188, kernel_initializer="normal", activation="relu", units=150)`
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", units=1)`
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 56865 samples, validate on 10035 samples
Epoch 1/200
56865/56865 [==============================] - 2s - loss: 55348696.6067 - val_loss: 49596182.8520
Epoch 2/200
56865/56865 [==============================] - 1s - loss: 45244700.6384 - val_loss: 41198808.9537
Epoch 3/200
56865/56865 [==============================] - 1s - loss: 40118256.9503 - val_loss: 37198436.7324
Epoch 4/200
56865/56865 [==============================] - 1s - loss: 35462653.0692 - val_loss: 31770426.5237
Epoch 5/200
56865/56865 [==============================] - 2s - loss: 29251021.0977 - val_loss: 24857340.0299
Epoch 6/200
56865/56865 [==============================] - 2s - loss: 22317472.5506 - val_loss: 17495676.9671
Epoch 7/200
56865/56865 [==============================] - 2s - loss: 15388143.6848 - val_loss: 11169319.5092
Epoch 8/200
56865/56865 [==============================] - 1s - loss: 10006191.3379 - val_loss: 6777096.3850
Epoch 9/200
56865/56865 [==============================] - 1s - loss: 6

56865/56865 [==============================] - 1s - loss: 3158893.1251 - val_loss: 2333659.2846
Epoch 76/200
56865/56865 [==============================] - 1s - loss: 3149185.4146 - val_loss: 2345642.2245
Epoch 77/200
56865/56865 [==============================] - 1s - loss: 3159473.4194 - val_loss: 2312394.1155
Epoch 78/200
56865/56865 [==============================] - 1s - loss: 3149429.0390 - val_loss: 2327497.2484
Epoch 79/200
56865/56865 [==============================] - 1s - loss: 3129515.5286 - val_loss: 2310244.7964
Epoch 80/200
56865/56865 [==============================] - 1s - loss: 3194233.7221 - val_loss: 2305430.1758
Epoch 81/200
56865/56865 [==============================] - 1s - loss: 3157342.8121 - val_loss: 2307687.486265800.51
Epoch 82/200
56865/56865 [==============================] - 1s - loss: 3171670.7269 - val_loss: 2294590.4095
Epoch 83/200
56865/56865 [==============================] - 1s - loss: 3140624.5910 - val_loss: 2297319.2830
Epoch 84/200
56865/56865

56865/56865 [==============================] - 1s - loss: 2825738.2662 - val_loss: 1953800.4101
Epoch 150/200
56865/56865 [==============================] - 1s - loss: 2811926.5582 - val_loss: 1962722.5254
Epoch 151/200
56865/56865 [==============================] - 1s - loss: 2807708.8846 - val_loss: 1948105.4188
Epoch 152/200
56865/56865 [==============================] - 1s - loss: 2799342.0332 - val_loss: 1945877.6725
Epoch 153/200
56865/56865 [==============================] - 1s - loss: 2773440.6771 - val_loss: 1939269.0896
Epoch 154/200
56865/56865 [==============================] - 1s - loss: 2796365.1032 - val_loss: 1936465.9361
Epoch 155/200
56865/56865 [==============================] - 1s - loss: 2748933.8331 - val_loss: 1928551.1755
Epoch 156/200
56865/56865 [==============================] - 1s - loss: 2757456.4387 - val_loss: 1921191.9406
Epoch 157/200
56865/56865 [==============================] - 2s - loss: 2784953.8425 - val_loss: 1917654.4350
Epoch 158/200
56865/5686

In [103]:
pred_test = model.predict(X_test)
pred_test = np.reshape(pred_test, pred_test.shape[0])
submission['Sales'] = pred_test
cols = ['Id','Sales']
submission['Id'] = submission.index + 1
submission = submission[cols]
submission.to_csv('submission.csv', index=False)